<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению. Сессия № 2
</center>
Автор материала: программист-исследователь Mail.ru Group, старший преподаватель Факультета Компьютерных Наук ВШЭ Юрий Кашницкий. Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center> Домашнее задание № 1
## <center> Анализ данных по сердечно-сосудистым заболеваниям

**В задании предлагается с помощью Pandas ответить на несколько вопросов по данным о сердечно-сосудистых заболеваниях (качать данные не надо, они уже есть в репозитории). Данные использовались в соревновании [Ml Boot Camp 5](http://mlbootcamp.ru/round/12/sandbox/).**

**Заполните код в клетках (где написано "Ваш код здесь") и ответьте на вопросы в [веб-форме](https://goo.gl/8D8uDf).**

В соревновании предлагалось определить наличие/отсутствие сердечно-сосудистых заболеваний (ССЗ) по результатам осмотра пациента.

**Описание данных.**

Объективные признаки:

 - Возраст (age)
 - Рост (height)
 - Вес (weight)
 - Пол (gender)
 

Результаты измерения:

 - Артериальное давление верхнее и нижнее (ap_hi, ap_lo)
 - Холестерин (cholesterol)
 - Глюкоза (gluc)
 

Субъективные признаки (со слов пациентов):

 - Курение (smoke)
 - Употребление алкоголя (alco)
 - Физическая активность (active)
 
Целевой признак (который интересно будет прогнозировать):
 - Наличие сердечно-сосудистых заболеваний по результатам классического врачебного осмотра (cardio)

Значения показателей холестерина и глюкозы представлены одним из трех классов: норма, выше нормы, значительно выше нормы. Значения субъективных признаков — бинарны.

Все показатели даны на момент осмотра.

Мы будем работать только с обучающей выборкой и с помощью `Pandas` проведем первичный анализ данных. 

Из библиотек нам понадобятся только `NumPy` и `Pandas`.

In [1]:
import warnings
warnings.simplefilter('ignore')

import numpy as np
import pandas as pd

Считываем данные из CSV-файла в объект pandas DataFrame.  

In [2]:
df = pd.read_csv('data/ml5/train.csv', sep=';', index_col='id')

Посмотрим не первые 5 записей.

In [3]:
df.head()

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
id,,,,,,,,,,,,
0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,17474,1,156,56.0,100,60,1,1,0,0,0,0


**<font color='red'>Вопрос 1 (1 балл).</font> Сколько мужчин и женщин представлено в этом наборе данных? Не было дано расшифровки признака "пол" (какому полу соответствует 1, а какому – 2 в признаке ``gender``) – это определите, посмотрев также на рост при разумном предположении, что в среднем мужчины выше.**

**<font color='red'>Варианты:</font>**
- 45530 женщин и 24470 мужчин
- 45530 мужчин и 24470 женщин
- 45470 женщин и 24530 мужчин
- 45470 мужчин и 24530 женщин

In [6]:
df.groupby('gender').agg({'height': {'mean': np.mean, 'count': len}})

height       
              mean  count
gender                   
1       161.355612  45530
2       169.947895  24470

**<font color='red'>Вопрос 2 (1 балл).</font> Кто в среднем чаще указывает, что употребляет алкоголь – мужчины или женщины?**

**<font color='red'>Варианты:</font>**
- женщины
- мужчины

In [7]:
df.groupby('gender').agg({'alco': np.mean})

,alco
gender,
1,0.025500
2,0.106375


**<font color='red'>Вопрос 3 (1 балл).</font> Во сколько раз (округленно) процент курящих среди мужчин больше, чем процент курящих среди женщин (по крайней мере, по этим анкетным данным)?**

**<font color='red'>Варианты:</font>**
- 4
- 8
- 12
- 16

In [8]:
r = df.groupby('gender').agg({'smoke': np.mean})
r.loc[2] / r.loc[1]

smoke    12.257833
dtype: float64

**<font color='red'>Вопрос 4 (1 балл).</font> Вы наверняка заметили, что значения возраста какие-то странные. Догадайтесь, в чем здесь измеряется возраст, и ответьте, на сколько месяцев (примерно) отличаются медианные значения возраста курящих и некурящих.**

**<font color='red'>Варианты:</font>**
- 5
- 10
- 15
- 20

In [9]:
from dateutil.relativedelta import relativedelta
from datetime import datetime

def map_age_field_to_birth_date(serial):
    seconds = (serial - 25569) * 86400.0
    return datetime.utcfromtimestamp(seconds)

def calc_age_in_years(born):
    return relativedelta(datetime.today(), born).years

def calc_age_in_months(born):
    rd = relativedelta(datetime.today(), born)
    return rd.years * 12 + rd.months


df['years'] = (df['age']
               .apply(map_age_field_to_birth_date)
               .apply(calc_age_in_years))
df['months'] = (df['age']
                .apply(map_age_field_to_birth_date)
                .apply(calc_age_in_months))

In [10]:
df.groupby('smoke').agg({'months': np.mean})

,months
smoke,
0,771.095988
1,784.711136


(88, 52)

**<font color='red'>Вопрос 5 (2 балла).</font> В [статье](https://ru.wikipedia.org/wiki/Сердечно-сосудистый_риск) на Википедии про сердечно-сосудистый риск показана шкала SCORE для расчёта риска смерти от сердечно-сосудистого заболевания в ближайшие 10 лет. Вот она:**
<img src='../../img/SCORE2007.png' width=70%>

Давайте посмотрим на правый верхний прямоугольник, отображающий сегмент курящих мужчин в возрасте от 60 до 65 лет. (Неочевидно, но тут для возраста и давления цифры означают верхнюю границу).

Видим 9-ку в левом нижнем углу этого прямоугольника и 47 – в правом верхнем. То есть если при этом систолическое (т.е. верхнее) артериальное давление – меньше 120 мм рт.ст., то риск ССЗ оценивается примерно в 5 раз ниже, чем если бы давление лежало в интервале [160, 180). 

Давайте посчитаем аналогичное значение, но на наших данных. 

Уточнения:
- Посчитайте признак ``age_years`` – возраст в годах, округлив до целых. Для данного примера отберите людей от 60 до 64 лет включительно
- Категории уровня холестрина на рисунке и в наших данных отличаются. Отображение значений на картинке в значения признака ``cholesterol`` следующее: 4 ммоль/л $\rightarrow$ 1, 5-7 ммоль/л $\rightarrow$ 2, 8 ммоль/л $\rightarrow$ 3.

Во сколько раз (округленно) отличаются доли больных в двух сегментах, описанных выше? Посчитайте на наших данных.

**<font color='red'>Варианты:</font>**
- 2
- 3
- 4
- 5

In [35]:
olds = df[(df['years'] >= 60) & (df['years'] <= 64)]
olds_low_risk = olds[(olds['cholesterol'] == 1) & (olds['ap_hi'] < 120)]
olds_high_risk = olds[(olds['cholesterol'] == 3) & ((olds['ap_lo'] >= 160) | (olds['ap_hi'] < 180))]

In [36]:
olds_low_risk.shape[0], olds_high_risk.shape[0]

(2000, 2101)

In [38]:
olds_high_risk[olds_high_risk['cardio'] == 1].shape[0] / olds_low_risk[olds_low_risk['cardio'] == 1].shape[0]

3.1254901960784314

**<font color='red'>Вопрос 6 (2 балла).</font> Постройте новый признак – BMI ([Body Mass Index](https://en.wikipedia.org/wiki/Body_mass_index)). Для этого надо вес в килограммах поделить на квадрат роста в метрах. Нормальными считаются значения BMI от 18.5 до 25. Выберите верные утверждения.**

**<font color='red'>Утверждения:</font>**
- Медианный BMI по выборке лежит в пределах нормы
- У женщин в среднем BMI выше, чем у мужчин
- У здоровых в среднем BMI выше, чем у больных
- В сегменте здоровых и непьющих мужчин BMI ближе к норме, чем в сегменте здоровых и непьющих женщин

In [43]:
df['meters'] = df['height'] / 100
df['bmi'] = df['weight'] / df['meters'] / df['meters']
df.head()

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio,years,months,meters,bmi
id,,,,,,,,,,,,,,,,
0,18393,2,168,62.0,110,80,1,1,0,0,1,0,67,807,1.68,21.967120
1,20228,1,156,85.0,140,90,3,1,0,0,1,1,62,747,1.56,34.927679
2,18857,1,165,64.0,130,70,3,1,0,0,0,1,66,792,1.65,23.507805
3,17623,2,169,82.0,150,100,1,1,0,0,1,1,69,833,1.69,28.710479
4,17474,1,156,56.0,100,60,1,1,0,0,0,0,69,838,1.56,23.011177


In [44]:
df['bmi'].mean()

27.55651255127882

In [45]:
df.groupby('gender').agg({'bmi': np.mean})

,bmi
gender,
1,27.987583
2,26.754442


In [46]:
df.groupby('cardio').agg({'bmi': np.mean})

,bmi
cardio,
0,26.548175
1,28.566061


In [47]:
df.groupby(['gender', 'alco']).agg({'bmi': np.mean})

bmi
gender alco           
1      0     27.941751
       1     29.739136
2      0     26.711539
       1     27.114857

**<font color='red'>Вопрос 7 (2 балла).</font> ** Можно заметить, что данные не особо-то чистые, много в них всякой "грязи" и неточностей. Еще лучше мы это увидим, когда обсудим визуализацию данных.

Отфильтруйте следующие сегменты пациентов (считаем это ошибками в данных)
- указанное нижнее значение артериального давления выше верхнего. 
- рост строго меньше 2.5%-перцентили или строго больше 97.5%-перцентили (используте `pd.Series.quantile`, если не знаете, что это такое – прочитайте)
- вес строго меньше 2.5%-перцентили или строго больше 97.5%-перцентили

Этот вовсе не вся чистка данных, которую можно было проделать, но пока остановимся на этом.

Сколько процентов данных (округленно) мы выбросили?

**<font color='red'>Варианты:</font>**
- 8
- 9
- 10
- 11

In [55]:
df_1 = df[df['ap_lo'] <= df['ap_hi']]
df_2 = df_1[(df_1['height'] >= df_1.height.quantile(0.025)) & (df_1['height'] <= df_1.height.quantile(0.975))]
df_3 = df_2[(df_2['weight'] >= df_2.weight.quantile(0.025)) & (df_2['weight'] <= df_2.weight.quantile(0.975))]

In [56]:
print(df.shape[0], df_1.shape[0], df_2.shape[0], df_3.shape[0])
print(100 - df_3.shape[0] / df.shape[0] * 100)

70000 68766 65859 62784
10.30857142857144
